In [55]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
# from torch.optim.lr_scheduler import ReduceLROnPlateau
import json
import time
from lscn import LSCNsClassifier

In [2]:
mydir = "../memory/cn/"
TRAIN_EN_X = mydir + "train_x.npy"
TRAIN_EN_Y = mydir + "train_y.npy"
VALID_EN_X = mydir + "dev_x.npy"
VALID_EN_Y = mydir + "dev_y.npy"

In [3]:
def load_phone_idx(file_path="../memory/cn/phone_idx.json"):
    with open(file_path, 'r', encoding='utf-8') as f:
        return json.load(f)

In [4]:
train_x = np.load(TRAIN_EN_X, allow_pickle=True)
train_y = np.load(TRAIN_EN_Y, allow_pickle=True)
valid_x = np.load(VALID_EN_X, allow_pickle=True)
valid_y = np.load(VALID_EN_Y, allow_pickle=True)
phone2idx = load_phone_idx()

### Debug

In [34]:
xx = train_x[:5]

In [41]:
embed = nn.Embedding(len(phone2idx), 128, padding_idx=0)

In [37]:
batch_x = [torch.as_tensor(x) for x in xx]

In [38]:
batch_x_length = [len(x) for x in xx]
batch_x_padded = pad_sequence(batch_x, batch_first=True)

In [39]:
batch_x_padded

tensor([[ 0,  1,  2,  1,  3,  1,  4,  5,  1,  2,  1,  6,  1,  0,  1,  7,  1,  8,
          1,  9,  1,  8, 10,  1, 11,  1, 12,  1,  7,  1, 13,  1,  2,  1,  3, 14,
          1, 12,  1, 15,  1,  2,  1, 12],
        [16,  1, 17,  1, 18,  1, 19,  1, 20, 21,  1, 22, 23, 24, 25,  1,  8,  1,
          4, 23, 26,  1,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [27,  1, 28,  1, 29,  1,  0,  1, 30,  1,  8,  1, 31,  1,  2,  1, 32,  1,
          4, 26,  1,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0],
        [16,  1,  8, 33,  1,  5, 34,  1, 35,  1, 26,  1,  8,  1,  2,  1, 30,  1,
          2,  1,  0,  1,  9,  1,  8,  1,  0,  1, 31,  1,  2,  1, 32,  1,  4, 36,
          1,  8,  1,  0,  0,  0,  0,  0],
        [ 4,  5,  1, 19,  1,  4,  1,  8, 10,  1, 37,  1, 12,  1, 13,  1,  2,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  

In [43]:
e = embed(batch_x_padded)

In [73]:
e[0]

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.0536,  0.5084,  1.6024,  ..., -1.2592,  0.6627, -0.3647],
        [ 0.0547,  0.2594,  1.6765,  ..., -0.2048, -0.1193, -1.2209],
        ...,
        [ 0.0547,  0.2594,  1.6765,  ..., -0.2048, -0.1193, -1.2209],
        [ 1.0536,  0.5084,  1.6024,  ..., -1.2592,  0.6627, -0.3647],
        [ 0.5330, -0.3109,  1.1353,  ..., -1.1916,  0.7405, -1.0822]],
       grad_fn=<SelectBackward>)

In [70]:
e.shape

torch.Size([5, 44, 128])

In [49]:
out = pack_padded_sequence(e, batch_x_length, batch_first=True, enforce_sorted=False)

In [50]:
lstm = nn.LSTM(128, 128, batch_first=True)

In [75]:
packed_out, (ht, ct) = lstm(out)

In [76]:
packed_out

PackedSequence(data=tensor([[ 0.0077, -0.0149,  0.0076,  ...,  0.0271, -0.0037,  0.0144],
        [ 0.0317,  0.0448, -0.1182,  ..., -0.0097, -0.1188,  0.2265],
        [ 0.0317,  0.0448, -0.1182,  ..., -0.0097, -0.1188,  0.2265],
        ...,
        [ 0.1409, -0.0490, -0.0069,  ..., -0.0665,  0.1408, -0.1145],
        [ 0.2239,  0.0357, -0.0847,  ..., -0.1123,  0.1913,  0.0676],
        [ 0.3641,  0.2215, -0.0767,  ...,  0.0058,  0.0688,  0.0374]],
       grad_fn=<CatBackward>), batch_sizes=tensor([5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1]), sorted_indices=tensor([0, 3, 1, 2, 4]), unsorted_indices=tensor([0, 2, 3, 1, 4]))

In [84]:
ht[-1].shape

torch.Size([5, 128])

In [79]:
ht.squeeze().shape

torch.Size([5, 128])

In [60]:
pout, _ = pad_packed_sequence(packed_out, batch_first=True)

In [80]:
pout.shape

torch.Size([5, 44, 128])

In [82]:
pout[:, -1, :].shape

torch.Size([5, 128])

In [85]:
pout[:, -1, :]

tensor([[ 3.6407e-01,  2.2152e-01, -7.6672e-02,  1.5911e-01,  1.3462e-01,
          9.0433e-02, -9.8943e-02, -2.9167e-01, -2.2048e-01, -6.1294e-03,
         -3.7788e-02,  8.9765e-03,  1.1675e-01, -3.8776e-03, -1.0426e-01,
         -7.3348e-02, -1.8457e-01, -2.1962e-02,  2.4640e-02, -8.3224e-04,
          2.3643e-01,  1.1367e-01,  1.0678e-01,  1.8088e-01,  4.2068e-02,
          8.6350e-02, -9.3287e-03, -2.0949e-01, -2.4873e-02, -2.6461e-01,
         -7.1012e-05, -1.0338e-01, -1.0874e-01,  2.2076e-02,  5.1201e-02,
          5.7082e-02, -8.6914e-02,  3.0095e-02, -1.1923e-02, -7.6101e-02,
         -3.2688e-03, -1.3786e-01, -1.9699e-01, -1.3920e-01,  3.2794e-02,
          1.9931e-02, -3.8614e-02, -5.2965e-02,  8.4016e-02, -4.7829e-02,
          6.5007e-02, -5.5391e-02,  1.5214e-01, -8.1094e-02, -1.7805e-01,
          1.8960e-01,  3.6045e-01,  3.6240e-03, -4.2896e-01, -6.2134e-02,
         -8.9225e-02, -2.9871e-01,  1.4109e-02, -3.4333e-02, -1.6012e-01,
          1.4299e-01,  2.8905e-01,  6.

### labels in english dataset are [0, 1, 2, 3, 4, 5, 8], change 8 to 6

In [19]:
max_label = np.max(train_y)
max_label
np.unique(valid_y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [5]:
# max_label = np.max(train_y)
# train_y[train_y == max_label] = len(np.unique(train_y)) - 1
# valid_y[valid_y == max_label] = len(np.unique(train_y)) - 1

In [20]:
class PhoneDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.length = len(Y)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, i):
        return self.X[i], self.Y[i]
    
    def collate_fn(batch):
        batch_x = [torch.as_tensor(x) for x, y in batch]
        batch_x_length = [len(x) for x, y in batch]
        batch_x_padded = pad_sequence(batch_x, batch_first=True)
        batch_y = torch.as_tensor([y for x, y in batch])
        return batch_x_padded, batch_x_length, batch_y

In [21]:
np.unique(train_y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [22]:
np.unique(valid_y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [23]:
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    total_predictions = 0
    correct_predictions = 0
    
    start_time = time.time()
    for batch_idx, (x_padded, x_lengths, target) in enumerate(train_loader):
        optimizer.zero_grad()
        print(x_lengths)
        x_padded = x_padded.to(device)
        target = target.to(device)
        outputs = model(x_padded, x_lengths)
        
        loss = criterion(outputs, target)
        _, predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    end_time = time.time()
    running_loss /= len(train_loader)
    accuracy = (correct_predictions/total_predictions) * 100.0
    print("Training loss: ", running_loss, "Time: ", end_time - start_time, 's')
    print("Training Accuracy", accuracy, "%")
    return running_loss, accuracy

In [24]:
def valid_epoch(model, valid_loader, criterion, device):
    start_time = time.time()
    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        total_predictions = 0
        correct_predictions = 0
    
        for batch_idx, (x_padded, x_lengths, target) in enumerate(valid_loader):
            x_padded = x_padded.to(device)
            target = target.to(device)
            outputs = model(x_padded, x_lengths)
            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()
            loss = criterion(outputs, target).detach()
            running_loss += loss.item()
    
    end_time = time.time()
    running_loss /= len(train_loader)
    accuracy = (correct_predictions/total_predictions) * 100.0
    print("Validation loss: ", running_loss, "Time: ", end_time - start_time, 's')
    print("Validation Accuracy", accuracy, "%")
    return running_loss, accuracy

In [25]:
train_dataset = PhoneDataset(train_x, train_y)
valid_dataset = PhoneDataset(valid_x, valid_y)

In [26]:
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=128, num_workers=8, collate_fn=PhoneDataset.collate_fn)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=128, collate_fn=PhoneDataset.collate_fn)

In [27]:
vocab_size =  len(phone2idx)
num_classes = len(np.unique(train_y))

In [28]:
model = LSCNsClassifier(vocab_size, num_classes)

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=5e-5)
# scheduler = ReduceLROnPlateau(optimizer, factor=0.3, patience=2, verbose=True)
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')
model.to(device)
print(model)

LSCNsClassifier(
  (embed): Embedding(78, 128, padding_idx=0)
  (convA): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (convB): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(256, 128, batch_first=True)
  (linear): Linear(in_features=128, out_features=9, bias=True)
)


In [30]:
num_epochs = 20

In [31]:
for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    valid_loss, valid_acc = valid_epoch(model, valid_loader, criterion, device)
#     scheduler.step(valid_loss)
    print("Epoch {} finished.".format(epoch))
    print('='*20)

[32, 12, 31, 12, 36, 37, 47, 41, 64, 34, 34, 29, 33, 39, 22, 29, 34, 41, 41, 32, 57, 62, 39, 54, 31, 72, 38, 30, 67, 32, 33, 23, 32, 29, 33, 55, 69, 39, 40, 16, 22, 56, 62, 41, 44, 23, 67, 25, 12, 29, 55, 79, 30, 29, 33, 23, 31, 21, 20, 33, 36, 19, 33, 35, 24, 29, 33, 39, 23, 49, 20, 38, 25, 25, 31, 40, 18, 46, 58, 25, 65, 49, 52, 101, 26, 1, 17, 47, 15, 17, 20, 25, 41, 6, 48, 23, 39, 7, 25, 34, 26, 27, 50, 25, 50, 24, 67, 26, 42, 33, 24, 27, 17, 23, 40, 34, 9, 12, 44, 46, 48, 33, 51, 36, 52, 52, 16, 13]
[30, 34, 15, 35, 27, 40, 36, 64, 63, 36, 47, 89, 86, 18, 20, 40, 28, 40, 14, 12, 40, 58, 55, 67, 26, 23, 51, 58, 14, 55, 56, 36, 18, 27, 31, 60, 56, 35, 14, 31, 62, 74, 28, 25, 12, 32, 33, 6, 35, 62, 45, 11, 25, 25, 1, 34, 45, 39, 12, 34, 38, 19, 7, 51, 20, 29, 79, 48, 37, 21, 25, 79, 38, 22, 36, 38, 17, 33, 39, 36, 19, 24, 33, 16, 44, 41, 39, 44, 35, 34, 24, 9, 6, 21, 48, 35, 49, 57, 43, 76, 1, 55, 15, 43, 34, 38, 68, 39, 65, 48, 17, 35, 33, 24, 42, 22, 31, 36, 42, 51, 24, 63, 44, 51,

[42, 51, 17, 64, 39, 32, 3, 54, 54, 51, 21, 100, 52, 59, 94, 40, 58, 84, 7, 62, 1, 36, 50, 80, 8, 64, 28, 32, 49, 40, 95, 51, 30, 50, 61, 34, 43, 45, 51, 29, 46, 29, 79, 49, 39, 107, 36, 33, 25, 46, 14, 36, 14, 46, 31, 22, 54, 16, 19, 44, 38, 48, 44, 35, 72, 20, 24, 17, 37, 64, 69, 10, 40, 1, 40, 56, 50, 54, 65, 13, 1, 54, 13, 23, 41, 27, 8, 57, 28, 49, 54, 33, 29, 74, 33, 58, 31, 49, 71, 41, 91, 47, 49, 24, 18, 44, 35, 28, 43, 32, 32, 77, 60, 80, 67, 23, 20, 33, 46, 29, 34, 12, 35, 51, 38, 65, 21, 56]
[39, 46, 51, 37, 8, 27, 25, 34, 69, 35, 9, 29, 10, 31, 32, 15, 21, 11, 7, 16, 27, 36, 17, 23, 91, 49, 50, 54, 77, 15, 22, 34, 34, 34, 29, 37, 27, 67, 75, 13, 42, 13, 52, 47, 42, 11, 44, 31, 10, 36, 36, 16, 17, 40, 5, 37, 25, 36, 26, 30, 24, 10, 45, 40, 24, 21, 59, 51, 37, 24, 25, 111, 44, 32, 24, 25, 18, 41, 58, 25, 78, 19, 30, 9, 25, 43, 51, 37, 46, 46, 38, 13, 15, 105, 52, 43, 33, 48, 31, 18, 30, 12, 23, 41, 75, 11, 11, 7, 47, 34, 35, 37, 21, 56, 87, 52, 26, 58, 59, 42, 14, 39, 49, 47,

Training loss:  2.1591536388840784 Time:  1.4440460205078125 s
Training Accuracy 52.318123511086675 %
Validation loss:  0.5008819380471873 Time:  0.06888222694396973 s
Validation Accuracy 43.86401326699834 %
Epoch 1 finished.
[41, 37, 27, 31, 26, 33, 31, 21, 60, 31, 22, 40, 23, 31, 20, 17, 83, 40, 7, 17, 71, 48, 37, 32, 74, 57, 31, 36, 77, 14, 30, 16, 21, 6, 8, 26, 46, 14, 34, 21, 53, 71, 29, 49, 52, 34, 24, 21, 10, 23, 47, 33, 26, 88, 38, 29, 19, 22, 27, 51, 48, 23, 59, 9, 46, 28, 37, 18, 15, 25, 42, 19, 21, 13, 52, 30, 30, 28, 55, 21, 8, 20, 42, 50, 8, 31, 48, 66, 20, 46, 54, 19, 17, 23, 54, 77, 73, 27, 17, 35, 32, 32, 25, 1, 22, 29, 56, 32, 32, 17, 37, 55, 50, 12, 37, 33, 35, 38, 19, 64, 34, 62, 57, 80, 22, 41, 89, 39]
[17, 29, 52, 70, 7, 12, 29, 49, 46, 11, 67, 83, 42, 41, 19, 37, 59, 38, 6, 44, 23, 12, 26, 48, 44, 14, 23, 27, 19, 44, 36, 67, 32, 31, 58, 13, 62, 44, 47, 15, 23, 53, 69, 48, 30, 100, 20, 33, 51, 22, 14, 18, 38, 36, 42, 27, 58, 19, 13, 89, 66, 34, 26, 54, 86, 24, 10, 

[27, 51, 85, 54, 31, 48, 40, 40, 9, 17, 7, 66, 12, 33, 47, 40, 58, 21, 27, 55, 37, 30, 19, 48, 9, 21, 13, 12, 77, 54, 20, 24, 24, 32, 69, 27, 76, 13, 68, 13, 39, 47, 68, 26, 18, 68, 15, 62, 24, 51, 23, 23, 24, 29, 14, 48, 32, 40, 11, 9, 31, 14, 7, 78, 26, 28, 8, 142, 54, 54, 15, 34, 91, 41, 55, 58, 25, 125, 23, 43, 33, 28, 21, 65, 11, 29, 31, 28, 25, 39, 25, 15, 35, 61, 51, 30, 16, 57, 35, 61, 36, 29, 50, 7, 73, 50, 25, 13, 48, 13, 3, 18, 29, 8, 47, 7, 25, 1, 8, 60, 7, 9, 25, 28, 55, 21, 31, 48]
[34, 49, 17, 65, 32, 53, 5, 24, 30, 38, 48, 47, 28, 40, 19, 29, 35, 18, 28, 27, 31, 12, 42, 29, 32, 39, 40, 33, 43, 26, 21, 13, 43, 57, 35, 51, 39, 51, 28, 31, 20, 26, 51, 15, 41, 5, 62, 11, 3, 41, 19, 27, 28, 54, 79, 22, 9, 74, 81, 42, 8, 55, 32, 47, 44, 18, 62, 21, 71, 49, 54, 29, 28, 49, 66, 18, 29, 81, 11, 52, 43, 67, 14, 31, 99, 16, 66, 40, 9, 34, 12, 37, 9, 7, 14, 69, 23, 14, 45, 37, 42, 75, 31, 55, 25, 25, 90, 32, 65, 97, 25, 18, 92, 71, 19, 80, 19, 45, 98, 37, 73, 23, 10, 35, 6, 8, 80, 

[51, 32, 7, 25, 30, 54, 29, 9, 37, 14, 92, 1, 29, 51, 61, 1, 8, 41, 66, 53, 42, 38, 51, 28, 22, 37, 39, 27, 41, 39, 14, 39, 35, 14, 59, 35, 54, 64, 44, 27, 45, 52, 31, 46, 45, 26, 62, 29, 48, 31, 52, 16, 23, 34, 15, 35, 97, 29, 34, 61, 18, 13, 32, 62, 42, 23, 44, 36, 28, 20, 23, 21, 54, 52, 15, 54, 15, 46, 34, 10, 38, 47, 56, 63, 53, 36, 22, 25, 26, 28, 61, 51, 35, 51, 28, 38, 33, 33, 22, 18, 46, 54, 46, 13, 38, 37, 53, 19, 13, 30, 24, 41, 18, 54, 9, 17, 22, 32, 30, 87, 34, 40, 35, 49, 16, 51, 30, 31]
[31, 46, 23, 77, 13, 40, 26, 42, 18, 84, 34, 21, 8, 102, 44, 34, 12, 22, 53, 58, 56, 34, 44, 14, 44, 5, 29, 8, 52, 47, 7, 32, 44, 76, 38, 36, 29, 7, 56, 56, 100, 98, 40, 40, 21, 39, 7, 40, 5, 54, 86, 35, 31, 57, 42, 42, 24, 18, 37, 21, 68, 32, 40, 7, 32, 23, 40, 45, 36, 36, 28, 56, 36, 46, 28, 58, 30, 58, 22, 62, 52, 106, 13, 61, 53, 11, 16, 55, 51, 28, 39, 26, 48, 8, 27, 63, 39, 38, 29, 95, 49, 27, 139, 49, 14, 3, 67, 26, 12, 28, 7, 32, 31, 28, 27, 61, 31, 5, 25, 27, 8, 41, 25, 42, 35, 3

...

[43, 20, 95, 54, 32, 37, 50, 48, 29, 39, 37, 45, 28, 52, 92, 75, 41, 8, 22, 20, 85, 42, 28, 32, 30, 13, 32, 31, 29, 36, 46, 65, 32, 33, 47, 41, 90, 64, 51, 38, 31, 39, 36, 36, 11, 26, 20, 19, 29, 41, 30, 51, 25, 16, 78, 40, 43, 12, 44, 88, 39, 81, 18, 49, 31, 23, 42, 73, 16, 61, 42, 34, 40, 20, 41, 47, 28, 33, 43, 40, 73, 30, 69, 41, 9, 35, 3, 41, 41, 9, 23, 53, 37, 12, 18, 21, 73, 25, 20, 65, 34, 52, 12, 22, 67, 66, 74, 8, 33, 43, 71, 66, 56, 52, 10, 39, 34, 39, 33, 37, 29, 25, 37, 19, 12, 48, 42, 45]
[70, 18, 37, 46, 60, 15, 18, 43, 21, 25, 26, 31, 86, 3, 44, 17, 92, 31, 44, 49, 56, 20, 39, 18, 11, 56, 49, 48, 39, 44, 28, 66, 54, 35, 25, 50, 47, 1, 36, 13, 30, 32, 7, 17, 61, 38, 47, 41, 16, 14, 34, 10, 26, 20, 51, 34, 50, 33, 74, 58, 58, 32, 15, 79, 33, 36, 62, 28, 30, 64, 34, 53, 29, 24, 18, 12, 12, 50, 32, 45, 38, 7, 30, 23, 12, 14, 27, 34, 19, 55, 7, 32, 37, 15, 69, 9, 17, 29, 39, 76, 33, 33, 16, 77, 30, 26, 17, 142, 31, 37, 49, 39, 31, 28, 19, 44, 38, 30, 65, 9, 42, 33, 31, 32, 7

[43, 21, 32, 18, 11, 16, 62, 58, 39, 18, 7, 32, 36, 46, 56, 32, 39, 62, 19, 36, 10, 38, 42, 28, 40, 41, 47, 31, 53, 13, 22, 13, 40, 29, 20, 161, 24, 91, 49, 30, 38, 62, 12, 38, 15, 45, 50, 54, 18, 51, 57, 18, 51, 51, 36, 31, 35, 72, 41, 30, 33, 41, 12, 42, 46, 82, 51, 16, 33, 14, 64, 8, 40, 22, 33, 85, 37, 27, 45, 3, 28, 58, 16, 49, 27, 41, 48, 11, 41, 53, 40, 47, 42, 36, 49, 29, 49, 29, 32, 56, 51, 48, 59, 28, 12, 43, 17, 30, 30, 23, 7, 136, 29, 31, 42, 39, 75, 41, 43, 53, 27, 56, 55, 8, 114, 29, 35, 48]
[5, 49, 66, 53, 30, 21, 37, 30, 39, 28, 28, 41, 24, 44, 37, 52, 16, 17, 45, 65, 10, 41, 28, 1, 48, 28, 36, 35, 42, 39, 66, 50, 68, 11, 7, 43, 25, 53, 24, 24, 91, 33, 30, 41, 28, 47, 60, 55, 23, 92, 29, 20, 49, 17, 23, 23, 61, 59, 5, 28, 10, 26, 25, 35, 36, 46, 21, 29, 48, 31, 48, 14, 21, 9, 36, 67, 34, 27, 15, 66, 36, 32, 40, 11, 31, 69, 7, 18, 89, 30, 31, 35, 19, 21, 63, 48, 98, 46, 51, 31, 40, 29, 25, 34, 29, 15, 58, 52, 25, 41, 43, 59, 57, 31, 19, 38, 48, 59, 44, 57, 61, 54, 35, 30

[42, 17, 10, 22, 13, 29, 51, 48, 37, 3, 85, 47, 14, 31, 30, 36, 45, 61, 34, 8, 36, 18, 37, 29, 5, 54, 41, 43, 85, 21, 30, 84, 24, 31, 37, 77, 48, 71, 23, 37, 106, 44, 10, 10, 70, 45, 84, 40, 29, 46, 17, 10, 63, 38, 7, 26, 45, 31, 26, 18, 100, 17, 35, 15, 23, 76, 42, 9, 29, 51, 44, 23, 22, 23, 38, 97, 32, 53, 43, 22, 20, 9, 15, 67, 61, 36, 62, 43, 27, 63, 36, 55, 55, 30, 10, 31, 44, 29, 54, 46, 26, 23, 19, 55, 31, 17, 55, 25, 41, 83, 23, 36, 7, 17, 20, 32, 9, 46, 37, 9, 6, 45, 33, 24, 28, 64, 17, 30]
[23, 21, 68, 27, 34, 55, 26, 15, 38, 52, 30, 53, 34, 34, 30, 13, 40, 21, 12, 26, 41, 29, 13, 60, 28, 71, 26, 35, 20, 24, 12, 19, 34, 34, 37, 13, 27, 34, 64, 34, 36, 87, 14, 44, 25, 28, 21, 61, 42, 1, 36, 15, 37, 26, 36, 49, 9, 12, 32, 16, 36, 42, 41, 43, 13, 30, 26, 63, 46, 46, 14, 61, 11, 62, 18, 44, 3, 25, 40, 30, 30, 30, 39, 56, 22, 55, 15, 13, 52, 12, 24, 42, 47, 59, 40, 19, 28, 13, 38, 64, 46, 76, 54, 39, 34, 34, 8, 33, 28, 57, 15, 53, 26, 15, 23, 55, 27, 48, 51, 20, 61, 34, 39, 11, 63

[1, 29, 40, 22, 40, 25, 32, 9, 54, 18, 23, 20, 55, 60, 66, 23, 105, 88, 29, 28, 59, 26, 6, 37, 40, 9, 39, 50, 21, 86, 5, 38, 21, 46, 56, 15, 45, 12, 17, 16, 9, 45, 68, 68, 41, 39, 8, 3, 36, 14, 16, 38, 32, 52, 31, 23, 27, 42, 24, 47, 43, 58, 42, 25, 14, 30, 44, 38, 57, 21, 45, 49, 107, 25, 22, 90, 28, 27, 35, 27, 6, 12, 7, 42, 37, 52, 49, 33, 25, 33, 35, 31, 45, 24, 18, 33, 63, 27, 12, 38, 37, 25, 40, 16, 31, 46, 25, 48, 20, 22, 39, 18, 15, 31, 67, 13, 46, 35, 42, 16, 22, 19, 31, 35, 38, 27, 25, 17]
[24, 40, 29, 14, 7, 29, 52, 35, 16, 42, 19, 26, 41, 33, 38, 8, 47, 21, 26, 17, 26, 47, 97, 50, 11, 78, 24, 40, 41, 15, 17, 46, 41, 17, 28, 50, 9, 48, 11, 39, 51, 9, 42, 42, 19, 46, 55, 15, 49, 57, 7, 15, 115, 54, 87, 19, 24, 18, 44, 67, 20, 39, 36, 37, 62, 32, 34, 51, 31, 15, 78, 44, 31, 31, 33, 60, 24, 43, 35, 54, 25, 33, 30, 20, 31, 56, 15, 44, 29, 34, 102, 32, 40, 41, 38, 30, 41, 16, 10, 32, 83, 41, 14, 50, 11, 60, 56, 33, 23, 66, 8, 97, 62, 21, 33, 49, 36, 31, 1, 58, 41, 105, 14, 34, 66

KeyboardInterrupt: 